+ Estável

In [1]:
import sqlite3
import pandas as pd
import unicodedata
import ollama
import os
from rapidfuzz import process
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama

# ==========================================
# AGENTE 1: ANALISTA SQL (Versão Final 3.0)
# ==========================================
class SQLAnalyst(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

    def preparar_agente(self, db_path):
        """Conecta e treina com regras de negócio blindadas contra erros de tipagem."""
        self.connect_to_sqlite(db_path)
        
        # Extração de metadados reais para o Fuzzy Match
        df_meta = self.run_sql("SELECT DISTINCT bairro, rua, especificacao FROM core_imovel")
        self.bairros = [str(x) for x in df_meta['bairro'].dropna().unique().tolist()]
        self.ruas = [str(x) for x in df_meta['rua'].dropna().unique().tolist()]
        self.tipos = [str(x) for x in df_meta['especificacao'].dropna().unique().tolist()]
        self.entidades = self.bairros + self.ruas + self.tipos

        if self.get_training_data().empty:
            # Treinamento de DDL (Baseado na estrutura real do db.sqlite3)
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN, -- 1 para Sim, 0 para Não
                especificacao VARCHAR(100) -- apartamento, casa, kitnet, studio, loft, cobertura
            );
            """)

            # Treinamento de Regras Críticas (Resolvendo falhas de auditoria)
            self.train(documentation=f"""
            - Localização: Juiz de Fora, MG.
            - REGRA DE ID: O campo 'id' é um INTEIRO. Ex: 'imóvel 131' deve ser traduzido como WHERE id = 131.
            - REGRA DE PETS: Se o cliente citar 'gato', 'cachorro' ou 'pets', use 'aceita_pets = 1'. 
            - NUNCA use LOWER() ou LIKE em colunas booleanas (aceita_pets) ou numéricas (preços, quartos, id).
            - Use LOWER() apenas para colunas de texto: bairro, rua, especificacao.
            - Custo Total = (preco_aluguel + preco_condominio + preco_iptu).
            - NUNCA adicione filtros de pet (aceita_pets = 0) a menos que o cliente peça 'que NÃO aceitem pets'.
            - Bairros em JF: {", ".join(self.bairros)}.
            """)

    def normalizar(self, texto):
        nfkd = unicodedata.normalize('NFKD', str(texto))
        return "".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

    def fuzzy_cleanup(self, pergunta):
        """Corrige a pergunta sem duplicar entidades ou alucinar bairros."""
        tokens = pergunta.split()
        resultado = []
        
        # Mapeamento rápido de tokens protegidos e numéricos
        for t in tokens:
            t_norm = self.normalizar(t)
            if t_norm.isdigit() or len(t_norm) <= 3:
                resultado.append(t)
                continue
            
            # Busca correspondência em bairros/ruas/tipos
            match = process.extractOne(t_norm, [self.normalizar(e) for e in self.entidades], score_cutoff=90)
            if match:
                # Recupera o nome original com a capitalização correta do banco
                idx = [self.normalizar(e) for e in self.entidades].index(match[0])
                entidade_real = self.entidades[idx]
                resultado.append(entidade_real)
            else:
                resultado.append(t)
        
        pergunta_limpa = " ".join(resultado)
        # Injeção semântica para Pets se houver menção a animais
        if any(x in pergunta.lower() for x in ["gato", "cachorro", "animal"]):
            pergunta_limpa += " que aceita pets"
            
        return pergunta_limpa

    def executar_consulta(self, pergunta):
        pergunta_limpa = self.fuzzy_cleanup(pergunta)
        try:
            sql = self.generate_sql(pergunta_limpa)
            df = self.run_sql(sql)
            return df, sql
        except Exception as e:
            return None, f"Erro: {str(e)}"

# ==========================================
# AGENTE 2: BIA (Persona Geofenced)
# ==========================================
class BiaPersona:
    def __init__(self, bairros_validos, model_name='deepseek-r1:8b'):
        self.model = model_name
        self.bairros_validos = bairros_validos
        self.system_prompt = f"""
        Você é a Bia, secretária virtual de uma imobiliária em Juiz de Fora.
        REGRAS:
        1. Se o banco de dados retornar 'Vazio', não invente dados. Diga que não encontrou e sugira bairros como: {", ".join(self.bairros_validos[:5])}.
        2. Nunca use termos técnicos de programação.
        3. Para cálculos, use os valores de aluguel, IPTU e condomínio fornecidos.
        """

    def responder(self, pergunta, df):
        contexto = df.to_dict(orient='records') if df is not None and not df.empty else "Nenhum imóvel encontrado."
        prompt = f"Pergunta do Cliente: {pergunta}\nDados Reais do Banco: {contexto}\nBia, responda:"
        
        try:
            response = ollama.generate(model=self.model, system=self.system_prompt, prompt=prompt, options={'temperature': 0.1})
            return response['response'].split("</thought>")[-1].strip()
        except Exception:
            return "Tive uma falha técnica rápida, mas posso pesquisar outro bairro para você em JF!"

# ==========================================
# MOTOR DE TESTES DE CONFERÊNCIA
# ==========================================
def bateria_de_conferencia(analista, bia):
    testes = [
        "Qual o custo total do imóvel 131?",              # Foco: Cálculo e ID Inteiro
        "Tem cobertura no Benfica que aceita gatos?",      # Foco: Regra de Pet Booleana
        "Quais casas tem no bairo Benfika?",              # Foco: Fuzzy Match sem alucinação
        "Qual o apartamento mais barato no Centro?"       # Foco: Ordenação e Filtro Geográfico
    ]
    
    print("\n📝 Iniciando Testes de Conferência Final...")
    for i, p in enumerate(testes, 1):
        df, sql = analista.executar_consulta(p)
        resposta = bia.responder(p, df)
        print(f"\n--- Teste {i} ---")
        print(f"Pergunta: {p}")
        print(f"SQL: {sql}")
        print(f"Bia: {resposta}")

if __name__ == "__main__":
    config_sql = {"model": "qwen2.5-coder:7b", "path": "./vanna_chroma_final_v3", "temperature": 0.0}
    analista = SQLAnalyst(config=config_sql)
    analista.preparar_agente("db.sqlite3")
    
    bia = BiaPersona(bairros_validos=analista.bairros)
    bateria_de_conferencia(analista, bia)


📝 Iniciando Testes de Conferência Final...
[{'role': 'system', 'content': "The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use responses to past questions also to guide you:\n\n\n            CREATE TABLE core_imovel (\n                id INTEGER PRIMARY KEY AUTOINCREMENT, \n                titulo VARCHAR(200), \n                descricao TEXT,\n                quartos INTEGER, \n                banheiros INTEGER, \n                garagem INTEGER, \n                area DECIMAL, \n                bairro VARCHAR(100), \n                rua VARCHAR(100), \n                preco_aluguel DECIMAL, \n                preco_iptu DECIMAL, \n                preco_condominio DECIMAL, \n                aceita_pets BOOLEAN, -- 1 para Sim, 0 para Não\n  

KeyboardInterrupt: 

INTERATIVO

In [ ]:
import sqlite3
import pandas as pd
import unicodedata
import ollama
import sys
from rapidfuzz import process
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama as VannaOllama

# ==============================================================================
# 1. ANALISTA SQL (O Código Robusto que você forneceu)
# ==============================================================================
class SQLAnalyst(ChromaDB_VectorStore, VannaOllama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        VannaOllama.__init__(self, config=config)
        self.bairros = []
        self.entidades = []

    def preparar_agente(self, db_path):
        """Conecta e treina com regras de negócio blindadas."""
        print("   [SQL] Conectando ao banco e carregando metadados...")
        self.connect_to_sqlite(db_path)
        
        try:
            # Extração de metadados reais para o Fuzzy Match
            df_meta = self.run_sql("SELECT DISTINCT bairro, rua, especificacao FROM core_imovel")
            self.bairros = [str(x) for x in df_meta['bairro'].dropna().unique().tolist()]
            ruas = [str(x) for x in df_meta['rua'].dropna().unique().tolist()]
            tipos = [str(x) for x in df_meta['especificacao'].dropna().unique().tolist()]
            self.entidades = self.bairros + ruas + tipos
        except Exception as e:
            print(f"   [SQL] Aviso: Não foi possível carregar metadados ({e}).")

        # Treinamento (Só treina se não tiver dados vetoriais)
        if self.get_training_data().empty:
            print("   [SQL] Realizando treinamento inicial do Vanna...")
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN, 
                especificacao VARCHAR(100) -- apartamento, casa, kitnet, studio, loft, cobertura
            );
            """)

            self.train(documentation=f"""
            - Localização: Juiz de Fora, MG.
            - A coluna de tipo de imóvel se chama 'especificacao'. NUNCA use 'tipo'.
            - REGRA DE PETS: Se o cliente citar 'gato', 'cachorro' ou 'pets', use 'aceita_pets = 1'. 
            - Use LOWER() apenas para colunas de texto: bairro, rua, especificacao.
            - Bairros em JF: {", ".join(self.bairros)}.
            """)

    def normalizar(self, texto):
        nfkd = unicodedata.normalize('NFKD', str(texto))
        return "".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

    def fuzzy_cleanup(self, pergunta):
        """Corrige a pergunta sem duplicar entidades ou alucinar bairros."""
        if not pergunta: return ""
        tokens = pergunta.split()
        resultado = []
        
        for t in tokens:
            t_norm = self.normalizar(t)
            if t_norm.isdigit() or len(t_norm) <= 3:
                resultado.append(t); continue
            
            # Busca correspondência exata ou aproximada
            match = process.extractOne(t_norm, [self.normalizar(e) for e in self.entidades], score_cutoff=88)
            if match:
                idx = [self.normalizar(e) for e in self.entidades].index(match[0])
                entidade_real = self.entidades[idx]
                resultado.append(entidade_real)
            else:
                resultado.append(t)
        
        pergunta_limpa = " ".join(resultado)
        if any(x in pergunta.lower() for x in ["gato", "cachorro", "animal"]):
            pergunta_limpa += " que aceita pets"
            
        return pergunta_limpa

    def executar_consulta(self, pergunta):
        pergunta_limpa = self.fuzzy_cleanup(pergunta)
        print(f"   [SQL] Query Processada: '{pergunta_limpa}'")
        try:
            sql = self.generate_sql(pergunta_limpa)
            
            # Validação simples
            if not sql or "SELECT" not in sql.upper():
                return None, "Não consegui gerar SQL válido."

            df = self.run_sql(sql)
            return df, sql
        except Exception as e:
            return None, f"Erro SQL: {str(e)}"

# ==============================================================================
# 2. BIA PERSONA (A "Boca" do Chatbot)
# ==============================================================================
class BiaPersona:
    def __init__(self, bairros_validos, model_name='llama3.1'):
        self.model = model_name
        self.bairros_validos = bairros_validos
        
    def responder(self, pergunta, df=None, historico=None):
        # Cria o contexto de dados
        if df is not None and not isinstance(df, str) and not df.empty:
            dados_str = df.to_string(index=False)
            contexto = f"RESULTADO DA BUSCA NO BANCO:\n{dados_str}\n(Use estes dados para responder. Se o usuário perguntar detalhes, olhe a tabela.)"
        elif isinstance(df, str):
            contexto = f"AVISO DO SISTEMA: {df}" # Caso de erro
        else:
            # Contexto vazio ou conversa fiada
            contexto = "Nenhum dado de imóvel novo. Apenas converse ou use o histórico."

        system_prompt = f"""
        Você é a Bia, secretária virtual de uma imobiliária em Juiz de Fora.
        
        INSTRUÇÕES:
        1. Se houver imóveis listados em 'RESULTADO DA BUSCA', apresente-os de forma resumida e simpática.
        2. Se o resultado for vazio, diga que não encontrou e sugira bairros: {", ".join(self.bairros_validos[:3])}.
        3. Se for apenas conversa ("Oi", "Obrigado"), seja breve e cordial.
        4. NÃO invente imóveis.
        """
        
        # Constrói o prompt final
        prompt_final = f"{system_prompt}\n\n{contexto}\n\nHistórico recente: {historico}\n\nUsuário: {pergunta}\nBia:"
        
        try:
            response = ollama.generate(model=self.model, prompt=prompt_final, options={'temperature': 0.3})
            return response['response']
        except Exception as e:
            return f"Desculpe, tive um erro técnico: {e}"

# ==============================================================================
# 3. O ORQUESTRADOR (O "Cérebro" que decide)
# ==============================================================================
class Orchestrator:
    def __init__(self, sql_agent, bia_persona):
        self.sql = sql_agent
        self.bia = bia_persona
        self.historico = [] # Memória simples
        self.ultimo_df = None # Memória de dados

    def classificar_intencao(self, texto):
        """
        Usa um modelo rápido para decidir se é SQL (Busca) ou CHAT.
        """
        prompt = f"""
        Classifique a frase do usuário em: BUSCA ou CHAT.
        
        Exemplos:
        "tem apartamento no centro?" -> BUSCA
        "quanto custa o aluguel?" -> BUSCA
        "Oi tudo bem?" -> CHAT
        "Obrigado" -> CHAT
        "Qual o endereço desse aí?" -> CHAT (Pois refere-se ao contexto anterior, não precisa de SQL novo)
        
        Frase: "{texto}"
        Responda APENAS a palavra (BUSCA ou CHAT).
        """
        try:
            # Usando temperatura 0 para ser determinístico
            resp = ollama.generate(model="qwen2.5-coder:7b", prompt=prompt, options={'temperature': 0.0})
            tag = resp['response'].strip().upper()
            if "BUSCA" in tag: return "BUSCA"
            return "CHAT"
        except:
            return "CHAT"

    def processar(self, texto):
        # 1. Identifica Intenção
        intencao = self.classificar_intencao(texto)
        print(f">>> [ROUTER] Intenção: {intencao}")

        dados_para_bia = None

        # 2. Executa Ação
        if intencao == "BUSCA":
            # Passa o texto ORIGINAL para o SQL Analyst (Sem alucinação de '2 quartos')
            df, sql_log = self.sql.executar_consulta(texto)
            self.ultimo_df = df
            dados_para_bia = df
        else:
            # Usa a memória anterior se for conversa sobre o imóvel
            dados_para_bia = self.ultimo_df

        # 3. Gera Resposta Final
        hist_str = "\n".join([f"{h['role']}: {h['content']}" for h in self.historico[-2:]])
        resposta = self.bia.responder(texto, df=dados_para_bia, historico=hist_str)

        # 4. Atualiza Histórico
        self.historico.append({'role': 'user', 'content': texto})
        self.historico.append({'role': 'assistant', 'content': resposta})
        
        return resposta

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    print("\n--- INICIANDO SISTEMA BIA v6 (Router + SQL Analyst Robusto) ---")
    
    # Configurações
    # QwenCoder é ótimo para SQL e Classificação Lógica
    config_sql = {"model": "qwen2.5-coder:7b", "path": "./vanna_chroma_final_v6"}
    
    # 1. Instancia o Especialista SQL (Seu código original)
    analista = SQLAnalyst(config=config_sql)
    analista.preparar_agente("db.sqlite3")
    
    # 2. Instancia a Persona (Llama 3.1 para falar bem)
    bia_persona = BiaPersona(bairros_validos=analista.bairros, model_name='llama3.1:8B')
    
    # 3. Instancia o Cérebro
    bot = Orchestrator(analista, bia_persona)
    
    print("\n✅ Sistema Pronto! (Sem alucinações de parâmetros)")
    
    while True:
        try:
            txt = input("\nVocê: ")
            if txt.lower() in ['sair', 'tchau', 'exit']:
                print("Bia: Tchau! Até logo.")
                break
                
            resp = bot.processar(txt)
            print(f"Bia: {resp}")
            
        except KeyboardInterrupt:
            break


--- INICIANDO SISTEMA BIA v6 (Router + SQL Analyst Robusto) ---
   [SQL] Conectando ao banco e carregando metadados...

✅ Sistema Pronto! (Sem alucinações de parâmetros)
>>> [ROUTER] Intenção: CHAT
Bia: Bom dia! Estou muito bem, obrigada! Como posso ajudar hoje? Você está procurando por um imóvel em Juiz de Fora?
>>> [ROUTER] Intenção: CHAT
Bia: Sim, temos alguns imóveis com cobertura no bairro do Benfica. Quer ver os detalhes?
>>> [ROUTER] Intenção: CHAT
Bia: Claro! Temos um lindo apartamento de 2 quartos e 1 banheiro, com área de lazer e cobertura no Benfica. O preço é muito acessível, R$ 250 mil. Quer saber mais detalhes?
>>> [ROUTER] Intenção: CHAT
Bia: Olá! Como posso ajudar hoje? Você está procurando por algo específico ou quer explorar nossas opções?
>>> [ROUTER] Intenção: CHAT
Bia: Olá! Estou aqui para te ajudar. Quais são suas necessidades atuais? Você está procurando um imóvel em Juiz de Fora?


em teste